Make a file of grid point distances and elevation biases
===

Import required modules
---

In [1]:
from netCDF4 import Dataset
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
from model import Model
from GCNet import GCNet

AWS metadata
---

In [2]:
A = GCNet("site_info.nc")
A.loadMeta()
nSites = A.getNSites()

WRF metadata
---

In [3]:
modelMetaFN = "wrf_geog.nc"
M = Model(modelMetaFN)
M.loadMeta()

2d dataset


Set up dataframe
---

In [4]:
df = pd.DataFrame(columns=['Site #', 'Site name', 'Site elev',\
                           'Grid Dist Closest', \
                           'Grid Elev Closest', 'Grid Elev Closest Bias', \
                           'Grid Elev Interp',  'Grid Elev Interp Bias'], \
                           index=range(nSites))

AWS info and interpolated elevation from old dataframe
---

In [5]:
dfInfoOld = pd.read_pickle('wrf_site_info.pkl' )
df['Site #'] = dfInfoOld['Site #']
df['Site name'] = dfInfoOld['Site name']
df['Site elev'] = dfInfoOld['Site elev']
df['Grid Elev Interp'] = dfInfoOld['ElevCorr']
df['Grid Elev Interp Bias'] = dfInfoOld['ElevErr']

Find closest points
---

In [6]:
closestDist = []
closestElev = []
closestBias = []
for awsIX in (range(nSites)):
    awsElev = A.getElev( awsIX )
    awsLatLon = ( A.getLat( awsIX ), A.getLon( awsIX ))
    closest = M.closestPoints( awsLatLon )
    d = closest[0]
    px = d['ixLon']
    py = d['ixLat']
    pdist = d['distance']
    plat  = M.getLat2d(py,px)
    plon  = M.getLon2d(py,px)
    pelev = M.getElev(py,px)
    perr = pelev - awsElev
    L = (awsIX, awsElev, pdist, plat, plon, pelev, perr)
#     print "%d %d: M: D = %.2f km, lat/lon/elev = %.2f/%.2f/%.2f m, elev_err = %.2f m" % L
    
    closestDist.append( pdist )
    closestElev.append( pelev )
    closestBias.append( perr )

df['Grid Dist Closest'] = closestDist
df['Grid Elev Closest'] = closestElev
df['Grid Elev Closest Bias'] = closestBias

In [7]:
df

,Site #,Site name,Site elev,Grid Dist Closest,Grid Elev Closest,Grid Elev Closest Bias,Grid Elev Interp,Grid Elev Interp Bias
0,1,Swiss Camp,1176,2.139199,1195.447998,19.447998,1163.27,-12.73
1,2,Crawford Point1,2022,5.580631,1969.975464,-52.024536,1988.95,-33.05
2,3,NASA-U,2334,6.709264,2379.932617,45.932617,2350.94,16.94
3,4,GITS,1869,6.423641,1863.167480,-5.832520,1885.53,16.53
4,5,Humboldt,1995,6.896669,1954.203857,-40.796143,1975.90,-19.10
5,6,Summit,3199,6.887268,3237.585449,38.585449,3242.02,43.02
6,7,Tunu-N,2052,4.154322,2115.738770,63.738770,2118.12,66.12
7,8,DYE-2,2099,3.756249,2103.551270,4.551270,2120.18,21.18
8,9,JAR1,932,7.666582,941.756165,9.756165,873.04,-58.96
9,10,Saddle,2467,5.605036,2470.074707,3.074707,2485.59,18.59


Create a netCDF file for the new data
-----

In [8]:
newNCFN = 'dist_elev_meta_wrf.nc'
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass

ncfile = Dataset(newNCFN,mode='w',format='NETCDF4_CLASSIC') 
ncfile.title = "Closest-point and interpolated grid point metadata"

stn_dim = ncfile.createDimension('station', nSites)

distVar = ncfile.createVariable('distanceClosest', np.float32, ('station'))
distVar.units = "km"
distVar.description = "Distance to nearest grid point"
distVar.long_name = "Distance to nearest grid point"
distVar[:] = list( df['Grid Dist Closest'].astype("float") )

elevClosestVar = ncfile.createVariable('elevationClosest', np.float32, ('station'))
elevClosestVar.units = "m"
elevClosestVar.description = "Elevation at nearest grid point"
elevClosestVar.long_name = "Elevation at nearest grid point"
elevClosestVar[:] = list( df['Grid Elev Closest'].astype("float") )

biasClosestVar = ncfile.createVariable('elevationBiasClosest', np.float32, ('station'))
biasClosestVar.units = "m"
biasClosestVar.description = "Elevation bias at nearest grid point"
biasClosestVar.long_name = "Elevation bias at nearest grid point"
biasClosestVar[:] = list( df['Grid Elev Closest Bias'].astype("float") )

elevInterpVar = ncfile.createVariable('elevationInterp', np.float32, ('station'))
elevInterpVar.units = "m"
elevInterpVar.description = "Interpolated elevation"
elevInterpVar.long_name = "Interpolated elevation"
elevInterpVar[:] = list( df['Grid Elev Interp'].astype("float") )

biasInterpVar = ncfile.createVariable('elevationBiasInterp', np.float32, ('station'))
biasInterpVar.units = "m"
biasInterpVar.description = "Interpolated elevation bias"
biasInterpVar.long_name = "Interpolated elevation bias"
biasInterpVar[:] = list( df['Grid Elev Interp Bias'].astype("float") )

ncfile.close()